In [1]:
import pandas as  pd
import re

In [2]:
df_0_5000 = pd.read_csv('data/match_result_0_5000.csv')
df_5001_10000 = pd.read_csv('data/match_result_5001_10000.csv')
df_10001_15000 = pd.read_csv('data/match_result_10001_15000.csv')
df_15001_22000 = pd.read_csv('data/match_result_15001_22000.csv')
df_22001_35000 = pd.read_csv('data/match_result_22001_35000.csv')
df_35001_40000 = pd.read_csv('data/match_result_35001_40000.csv')

In [4]:
df_0_5000['info'][0]

"{'gameCreation': 1692279451325, 'gameDuration': 913, 'gameEndTimestamp': 1692280375722, 'gameId': 6657198842, 'gameMode': 'CLASSIC', 'gameName': 'teambuilder-match-6657198842', 'gameStartTimestamp': 1692279462096, 'gameType': 'MATCHED_GAME', 'gameVersion': '13.16.525.6443', 'mapId': 11, 'participants': [{'allInPings': 0, 'assistMePings': 0, 'assists': 0, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 112, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 0, 'baronTakedowns': 0, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 0, 'completeSupportQuestInTime': 0, 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.4111534174022164, 'controlWardsPlaced': 1, 'damagePerMinute': 469.5220916122018, 'damageTakenOnTeamPercentage': 0.2522669434432903, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 3, 'dodgeSkillShotsSmallWindow': 0, 'doubleAces': 0, 'dragonTakedowns': 0, 'earlyLaningPhaseGoldExpAd

In [110]:
df = df[df['info'].notnull()]

In [112]:
df.reset_index(drop=True, inplace=True)

In [113]:
pattern_time = r"'timePlayed': ([0-9]*)"
# pattern_duration = r"'gameDuration': ([0-9]*)"

for i in range(len(df_35001_40000)):
    info_str = df_35001_40000['info'][i]
    matches_time = re.findall(pattern_time, info_str)
#     matches_duration = re.findall(pattern_duration, df_35001_40000[i])
    if len(list(set(matches_time))) != 1:
        print(i)

In [99]:
pattern_time = r"'timePlayed': ([0-9]*)"
matches_time = re.findall(pattern_time, df_35001_40000['info'][0])

matches_time

['821', '821', '821', '821', '821', '821', '821', '821']

In [29]:
df_22001_35000['info'].isnull().sum()

18

In [24]:
def teams_preprocessing(df, i):
    
    # first kill 목록
    tf = ['blue_baron_first', 'blue_champion_first', 'blue_dragon_first', 'blue_inhibitor_first', 'blue_riftherald_first', 'blue_tower_first', 'blue_win',
              'red_baron_first', 'red_champion_first', 'red_dragon_first', 'red_inhibitor_first', 'red_riftherald_first', 'red_tower_first', 'red_win']

    # total kill 목록
    kill = ['blue_baron', 'blue_champion', 'blue_dragon', 'blue_inhibitor', 'blue_riftherald', 'blue_tower',
            'red_baron', 'red_champion', 'red_dragon', 'red_inhibitor', 'red_riftherald', 'red_tower']

    # 기타 컬럼 목록
    others = ['blue_lane_minion', 'red_lane_minion', 'blue_lane_gold', 'red_lane_gold', 'duration', 'blue_gold_perM', 'red_gold_perM', 'blue_gold_earned', 
             'red_gold_earned', 'blue_kda', 'red_kda', 'blue_solo_kill', 'red_solo_kill', 'blue_visionscore_perM', 'red_visionscore_perM', 'blue_level', 
             'red_level', 'blue_death', 'red_death', 'blue_damage', 'red_damage', 'blue_jungle_minion', 'red_jungle_minion', 'blue_total_minion',
             'red_total_minion', 'blue_death_time', 'red_death_time', 'blue_visionscore', 'red_visionscore']

    # 추가: df['info'][i] 타입 확인
    if pd.isna(df['info'][i]):
        print(f"Skipping index {i}")
        return None  # 혹은 빈 DataFrame 반환
    

    # teams 데이터 불러오기
    teams = df['info'][i][df['info'][i].find('teams'):]

    # first kill
    pattern_tf = r'(True|False)'
    matches_tf = re.findall(pattern_tf, teams)

    # total kill
    pattern_kills = r"'kills': ([0-9]*)"
    matches_kill = re.findall(pattern_kills, teams)

    # laneMinionsFirst10Minutes
    pattern_laneminion = r"'laneMinionsFirst10Minutes': ([0-9]*)"
    matches_laneminion = re.findall(pattern_laneminion, df['info'][i])
    matches_laneminion = [sum(map(int, matches_laneminion[:5])), sum(map(int, matches_laneminion[5:]))]

    # laningPhaseGoldExpAdvantage
    pattern_lanegold = r"'laningPhaseGoldExpAdvantage': ([0-9]*)"
    matches_lanegold = re.findall(pattern_lanegold, df['info'][i])
    matches_lanegold = [sum(map(int, matches_lanegold[:5])), sum(map(int, matches_lanegold[5:]))]

    # gameDuration
    pattern_gameduration = r"'gameDuration': ([0-9]*)"
    matches_gameduration = re.findall(pattern_gameduration, df['info'][i])
    matches_gameduration = list(map(int, matches_gameduration))

    # golPerMinute
    pattern_gold_perM = r"'goldPerMinute': ([0-9]*\.?[0-9]*)"
    matches_gold_perM = re.findall(pattern_gold_perM, df['info'][i])
    matches_gold_perM = [round(sum(map(float, matches_gold_perM[:5]))/5, 2), round(sum(map(float, matches_gold_perM[5:]))/5, 2)]

    # goldEarned
    pattern_goldearned = r"'goldEarned': ([0-9]*)"
    matches_goldearned = re.findall(pattern_goldearned, df['info'][i])
    matches_goldearned = [sum(map(int, matches_goldearned[:5])), sum(map(int, matches_goldearned[5:]))]

    # kda
    pattern_kda = r"'kda': ([0-9]*\.?[0-9]*)"
    matches_kda = re.findall(pattern_kda, df['info'][i])
    matches_kda = [round(sum(map(float, matches_kda[:5]))/5, 2), round(sum(map(float, matches_kda[5:]))/5, 2)]

    # soloKills
    pattern_solokill = r"'soloKills': ([0-9]*)"
    matches_solokill = re.findall(pattern_solokill, df['info'][i])
    matches_solokill = [sum(map(int, matches_solokill[:5])), sum(map(int, matches_solokill[5:]))]

    # visionScorePerMinute
    pattern_visionScore_perM = r"'visionScorePerMinute': ([0-9]*\.?[0-9]*)"
    matches_visionScore_perM = re.findall(pattern_visionScore_perM, df['info'][i])
    matches_visionScore_perM = [round(sum(map(float, matches_visionScore_perM[:5]))/5, 2), round(sum(map(float, matches_visionScore_perM[5:]))/5, 2)]

    # champLevel
    pattern_champLevel = r"'champLevel': ([0-9]*)"
    matches_champLevel = re.findall(pattern_champLevel, df['info'][i])
    matches_champLevel = [sum(map(int, matches_champLevel[:5]))/5, sum(map(int, matches_champLevel[5:]))/5]

    # deaths
    pattern_deaths = r"'deaths': ([0-9]*)"
    matches_deaths = re.findall(pattern_deaths, df['info'][i])
    matches_deaths = [sum(map(int, matches_deaths[:5])), sum(map(int, matches_deaths[5:]))]

    # totalDamageDealtToChampions
    pattern_damage = r"'totalDamageDealtToChampions': ([0-9]*)"
    matches_damage = re.findall(pattern_damage, df['info'][i])
    matches_damage = [sum(map(int, matches_damage[:5]))/5, sum(map(int, matches_damage[5:]))/5]

    # totalEnemyJungleMinionsKilled
    pattern_jungle_minion = r"'totalEnemyJungleMinionsKilled': ([0-9]*)"
    matches_jungle_minion = re.findall(pattern_jungle_minion, df['info'][i])
    matches_jungle_minion = [sum(map(int, matches_jungle_minion[:5])), sum(map(int, matches_jungle_minion[5:]))]

    # totalMinionsKilled
    pattern_total_minion = r"'totalMinionsKilled': ([0-9]*)"
    matches_total_minion = re.findall(pattern_total_minion, df['info'][i])
    matches_total_minion = [sum(map(int, matches_total_minion[:5])), sum(map(int, matches_total_minion[5:]))]

    # totalTimeSpentDead
    pattern_death_time = r"'totalTimeSpentDead': ([0-9]*)"
    matches_death_time = re.findall(pattern_death_time, df['info'][i])
    matches_death_time = [sum(map(int, matches_death_time[:5])), sum(map(int, matches_death_time[5:]))]

    # visionScore
    pattern_visionscore = r"'visionScore': ([0-9]*)"
    matches_visionscore = re.findall(pattern_visionscore, df['info'][i])
    matches_visionscore = [sum(map(int, matches_visionscore[:5])), sum(map(int, matches_visionscore[5:]))]

    # 데이터 합치기
    combined_data = dict(zip(tf + kill + others, matches_tf + matches_kill + matches_laneminion + matches_lanegold + 
                             matches_gameduration + matches_gold_perM + matches_goldearned + matches_kda + matches_solokill + matches_visionScore_perM +
                             matches_champLevel + matches_deaths + matches_damage + matches_jungle_minion + matches_total_minion + matches_death_time +
                             matches_visionscore))

    # 데이터 프레임
    match_history = pd.DataFrame([combined_data])
    return match_history

all_match_history = pd.DataFrame()

for i in range(len(df_0_5000)):
    result = teams_preprocessing(df_0_5000, i)
    if result is not None:  # 추가: 반환된 값이 None이 아닐 경우만 concat
        all_match_history = pd.concat([all_match_history, result])

Skipping index 4044
Skipping index 4170
Skipping index 4318


In [25]:
all_match_history

""


In [13]:
all_match_history.to_excel('all_match_history_15001_22000.xlsx')

In [15]:
df_1 = pd.read_excel('data/all_match_history_22001_35000.xlsx')
df_2 = pd.read_excel('data/all_match_history_35000_40000.xlsx')
df_3 = pd.read_excel('data/all_match_history_0_5000.xlsx')
df_4 = pd.read_excel('data/all_match_history_5001_10000.xlsx')
df_5 = pd.read_excel('data/all_match_history_10001_15000.xlsx')
df_6 = pd.read_excel('data/all_match_history_15001_22000.xlsx')

In [22]:
total_df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6])

In [25]:
total_df

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,False,False,True,True,True,True,True,False,True,False,...,19954.8,13585.2,38,0,584,497,586,910,112,99
1,False,True,True,False,False,False,False,True,False,False,...,26190.0,27490.8,22,19,715,723,1528,1319,207,194
2,True,True,True,True,True,True,True,False,False,False,...,19816.8,10824.4,61,13,697,661,360,830,199,155
3,False,False,True,False,False,True,True,False,True,False,...,10525.4,7117.4,2,8,469,395,229,500,76,82
4,False,False,False,False,True,True,True,False,True,True,...,7055.6,7238.2,15,12,452,425,212,255,80,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6987,False,False,False,False,False,False,False,False,True,True,...,6451.8,10501.6,0,8,425,473,454,156,69,103
6988,True,True,True,True,True,True,True,False,False,False,...,15966.8,9784.6,65,10,525,453,413,788,111,100
6989,False,False,False,False,True,True,False,True,True,True,...,27253.0,29899.8,20,24,727,623,1452,1127,170,194
6990,True,False,True,True,True,False,True,False,True,False,...,16478.2,14236.6,4,12,574,599,578,820,153,121


In [24]:
total_df = total_df.drop('Unnamed: 0', axis=1)

In [26]:
total_df.to_csv('data/all_match_history_0_40000.csv', index=False, encoding='utf-8')

In [29]:
df = pd.read_csv('data/all_match_history_0_40000.csv', dtype='str')

In [30]:
df

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,False,False,True,True,True,True,True,False,True,False,...,19954.8,13585.2,38,0,584,497,586,910,112,99
1,False,True,True,False,False,False,False,True,False,False,...,26190.0,27490.8,22,19,715,723,1528,1319,207,194
2,True,True,True,True,True,True,True,False,False,False,...,19816.8,10824.4,61,13,697,661,360,830,199,155
3,False,False,True,False,False,True,True,False,True,False,...,10525.4,7117.4,2,8,469,395,229,500,76,82
4,False,False,False,False,True,True,True,False,True,True,...,7055.6,7238.2,15,12,452,425,212,255,80,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39952,False,False,False,False,False,False,False,False,True,True,...,6451.8,10501.6,0,8,425,473,454,156,69,103
39953,True,True,True,True,True,True,True,False,False,False,...,15966.8,9784.6,65,10,525,453,413,788,111,100
39954,False,False,False,False,True,True,False,True,True,True,...,27253.0,29899.8,20,24,727,623,1452,1127,170,194
39955,True,False,True,True,True,False,True,False,True,False,...,16478.2,14236.6,4,12,574,599,578,820,153,121


In [31]:
df_1 = pd.read_csv('data/all_match_history_0_40000.csv', dtype=str)
df_2 = pd.read_csv('data/Combined_History.csv', dtype=str)
df_3 = pd.read_csv('data/match_history_80001.csv', dtype=str)

In [32]:
df_1

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,False,False,True,True,True,True,True,False,True,False,...,19954.8,13585.2,38,0,584,497,586,910,112,99
1,False,True,True,False,False,False,False,True,False,False,...,26190.0,27490.8,22,19,715,723,1528,1319,207,194
2,True,True,True,True,True,True,True,False,False,False,...,19816.8,10824.4,61,13,697,661,360,830,199,155
3,False,False,True,False,False,True,True,False,True,False,...,10525.4,7117.4,2,8,469,395,229,500,76,82
4,False,False,False,False,True,True,True,False,True,True,...,7055.6,7238.2,15,12,452,425,212,255,80,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39952,False,False,False,False,False,False,False,False,True,True,...,6451.8,10501.6,0,8,425,473,454,156,69,103
39953,True,True,True,True,True,True,True,False,False,False,...,15966.8,9784.6,65,10,525,453,413,788,111,100
39954,False,False,False,False,True,True,False,True,True,True,...,27253.0,29899.8,20,24,727,623,1452,1127,170,194
39955,True,False,True,True,True,False,True,False,True,False,...,16478.2,14236.6,4,12,574,599,578,820,153,121


In [33]:
df_2

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,False,False,True,True,True,True,True,True,True,False,...,23840.0,21317.6,24.0,8.0,647.0,655.0,524.0,1082.0,198.0,186.0
1,False,True,True,False,True,True,True,False,False,False,...,7991.4,4736.2,34.0,8.0,377.0,293.0,85.0,384.0,77.0,55.0
2,False,False,False,True,False,True,True,False,True,False,...,19165.6,13407.2,0.0,0.0,161.0,150.0,638.0,734.0,0.0,0.0
3,False,True,True,True,True,True,True,False,False,False,...,9218.0,5902.6,10.0,0.0,516.0,492.0,108.0,370.0,104.0,73.0
4,True,False,True,False,False,False,True,False,True,False,...,22453.8,15994.6,49.0,14.0,679.0,586.0,742.0,1058.0,182.0,138.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34941,False,False,False,True,True,True,False,True,True,True,...,27742.8,28527.4,44.0,10.0,805.0,745.0,1304.0,1058.0,219.0,210.0
34942,True,False,True,True,False,False,True,False,True,False,...,20727.2,23997.0,16.0,29.0,692.0,748.0,901.0,971.0,224.0,195.0
34943,True,False,True,True,False,False,False,False,True,False,...,25507.6,24062.6,53.0,28.0,736.0,833.0,947.0,1205.0,305.0,214.0
34944,False,False,True,True,True,True,True,False,True,False,...,31278.6,22598.0,70.0,26.0,663.0,659.0,989.0,1640.0,175.0,157.0


In [34]:
df_3

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,True,True,True,True,True,False,True,False,False,False,...,14875.0,10679.0,39.0,24.0,513.0,575.0,382.0,650.0,133.0,128.0
1,False,True,False,False,False,False,True,True,False,True,...,28307.4,30100.2,0.0,36.0,772.0,679.0,1385.0,1714.0,204.0,203.0
2,False,False,False,False,True,True,False,True,True,True,...,19043.4,15597.4,40.0,21.0,640.0,569.0,741.0,566.0,172.0,168.0
3,True,True,False,True,True,True,True,False,False,True,...,18689.2,15785.8,58.0,4.0,912.0,910.0,528.0,878.0,243.0,256.0
4,False,False,True,False,True,False,False,True,True,False,...,8715.6,13911.8,0.0,50.0,567.0,611.0,731.0,230.0,130.0,133.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34337,False,False,True,False,True,True,True,False,True,False,...,6395.8,3533.6,8,0,442,323,66,275,85,38
34338,False,True,False,True,False,True,True,False,False,True,...,18231.4,8992.6,10,8,589,494,313,674,119,113
34339,False,True,False,False,False,True,False,True,False,True,...,11571.4,14072.4,15,51,608,631,789,367,104,138
34340,False,True,False,True,True,True,True,False,False,False,...,11078.8,2020.4,4,0,245,36,10,570,3,0


In [35]:
total_df = pd.concat([df_1, df_2, df_3])

In [36]:
total_df

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,False,False,True,True,True,True,True,False,True,False,...,19954.8,13585.2,38,0,584,497,586,910,112,99
1,False,True,True,False,False,False,False,True,False,False,...,26190.0,27490.8,22,19,715,723,1528,1319,207,194
2,True,True,True,True,True,True,True,False,False,False,...,19816.8,10824.4,61,13,697,661,360,830,199,155
3,False,False,True,False,False,True,True,False,True,False,...,10525.4,7117.4,2,8,469,395,229,500,76,82
4,False,False,False,False,True,True,True,False,True,True,...,7055.6,7238.2,15,12,452,425,212,255,80,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34337,False,False,True,False,True,True,True,False,True,False,...,6395.8,3533.6,8,0,442,323,66,275,85,38
34338,False,True,False,True,False,True,True,False,False,True,...,18231.4,8992.6,10,8,589,494,313,674,119,113
34339,False,True,False,False,False,True,False,True,False,True,...,11571.4,14072.4,15,51,608,631,789,367,104,138
34340,False,True,False,True,True,True,True,False,False,False,...,11078.8,2020.4,4,0,245,36,10,570,3,0


In [72]:
total_df = total_df.replace({True: 1, False: 0})

In [37]:
total_df = total_df.replace({'True': 1, 'False': 0})

In [38]:
total_df

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,0,0,1,1,1,1,1,0,1,0,...,19954.8,13585.2,38,0,584,497,586,910,112,99
1,0,1,1,0,0,0,0,1,0,0,...,26190.0,27490.8,22,19,715,723,1528,1319,207,194
2,1,1,1,1,1,1,1,0,0,0,...,19816.8,10824.4,61,13,697,661,360,830,199,155
3,0,0,1,0,0,1,1,0,1,0,...,10525.4,7117.4,2,8,469,395,229,500,76,82
4,0,0,0,0,1,1,1,0,1,1,...,7055.6,7238.2,15,12,452,425,212,255,80,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34337,0,0,1,0,1,1,1,0,1,0,...,6395.8,3533.6,8,0,442,323,66,275,85,38
34338,0,1,0,1,0,1,1,0,0,1,...,18231.4,8992.6,10,8,589,494,313,674,119,113
34339,0,1,0,0,0,1,0,1,0,1,...,11571.4,14072.4,15,51,608,631,789,367,104,138
34340,0,1,0,1,1,1,1,0,0,0,...,11078.8,2020.4,4,0,245,36,10,570,3,0


In [39]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109245 entries, 0 to 34341
Data columns (total 55 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   blue_baron_first       109245 non-null  object
 1   blue_champion_first    109245 non-null  object
 2   blue_dragon_first      109245 non-null  object
 3   blue_inhibitor_first   109245 non-null  object
 4   blue_riftherald_first  109245 non-null  object
 5   blue_tower_first       109245 non-null  object
 6   blue_win               109245 non-null  object
 7   red_baron_first        109245 non-null  object
 8   red_champion_first     109245 non-null  object
 9   red_dragon_first       109245 non-null  object
 10  red_inhibitor_first    109245 non-null  object
 11  red_riftherald_first   109245 non-null  object
 12  red_tower_first        109245 non-null  object
 13  red_win                109245 non-null  object
 14  blue_baron             109245 non-null  object
 15  b

In [40]:
total_df.isnull().sum()

blue_baron_first         0
blue_champion_first      0
blue_dragon_first        0
blue_inhibitor_first     0
blue_riftherald_first    0
blue_tower_first         0
blue_win                 0
red_baron_first          0
red_champion_first       0
red_dragon_first         0
red_inhibitor_first      0
red_riftherald_first     0
red_tower_first          0
red_win                  0
blue_baron               0
blue_champion            0
blue_dragon              0
blue_inhibitor           0
blue_riftherald          0
blue_tower               0
red_baron                0
red_champion             0
red_dragon               0
red_inhibitor            0
red_riftherald           0
red_tower                0
blue_lane_minion         0
red_lane_minion          0
blue_lane_gold           0
red_lane_gold            4
duration                 4
blue_gold_perM           4
red_gold_perM            4
blue_gold_earned         4
red_gold_earned          4
blue_kda                 4
red_kda                  4
b

In [44]:
total_df = total_df[total_df['red_lane_gold'].notnull()]

In [57]:
final_df = total_df.copy()

In [58]:
numeric_columns = list(total_df.columns)

for col in numeric_columns:
    final_df[col] = pd.to_numeric(final_df[col])

In [61]:
final_df

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,0,0,1,1,1,1,1,0,1,0,...,19954.8,13585.2,38.0,0.0,584.0,497.0,586.0,910.0,112.0,99.0
1,0,1,1,0,0,0,0,1,0,0,...,26190.0,27490.8,22.0,19.0,715.0,723.0,1528.0,1319.0,207.0,194.0
2,1,1,1,1,1,1,1,0,0,0,...,19816.8,10824.4,61.0,13.0,697.0,661.0,360.0,830.0,199.0,155.0
3,0,0,1,0,0,1,1,0,1,0,...,10525.4,7117.4,2.0,8.0,469.0,395.0,229.0,500.0,76.0,82.0
4,0,0,0,0,1,1,1,0,1,1,...,7055.6,7238.2,15.0,12.0,452.0,425.0,212.0,255.0,80.0,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34337,0,0,1,0,1,1,1,0,1,0,...,6395.8,3533.6,8.0,0.0,442.0,323.0,66.0,275.0,85.0,38.0
34338,0,1,0,1,0,1,1,0,0,1,...,18231.4,8992.6,10.0,8.0,589.0,494.0,313.0,674.0,119.0,113.0
34339,0,1,0,0,0,1,0,1,0,1,...,11571.4,14072.4,15.0,51.0,608.0,631.0,789.0,367.0,104.0,138.0
34340,0,1,0,1,1,1,1,0,0,0,...,11078.8,2020.4,4.0,0.0,245.0,36.0,10.0,570.0,3.0,0.0


In [60]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109241 entries, 0 to 34341
Data columns (total 55 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   blue_baron_first       109241 non-null  int64  
 1   blue_champion_first    109241 non-null  int64  
 2   blue_dragon_first      109241 non-null  int64  
 3   blue_inhibitor_first   109241 non-null  int64  
 4   blue_riftherald_first  109241 non-null  int64  
 5   blue_tower_first       109241 non-null  int64  
 6   blue_win               109241 non-null  int64  
 7   red_baron_first        109241 non-null  int64  
 8   red_champion_first     109241 non-null  int64  
 9   red_dragon_first       109241 non-null  int64  
 10  red_inhibitor_first    109241 non-null  int64  
 11  red_riftherald_first   109241 non-null  int64  
 12  red_tower_first        109241 non-null  int64  
 13  red_win                109241 non-null  int64  
 14  blue_baron             109241 non-nul

In [62]:
final_df = final_df.astype({'blue_baron':'int64', 'blue_champion':'int64', 'blue_dragon':'int64', 'blue_tower':'int64',
                'red_baron':'int64', 'red_lane_gold':'int64', 'duration':'int64', 'blue_gold_earned':'int64',
                'red_gold_earned':'int64', 'blue_solo_kill':'int64', 'red_solo_kill':'int64', 'blue_death': 'int64',
                'red_death':'int64', 'blue_jungle_minion':'int64', 'red_jungle_minion':'int64', 'blue_total_minion':'int64',
                'red_total_minion':'int64', 'blue_death_time':'int64', 'red_death_time':'int64', 'blue_visionscore':'int64',
                'red_visionscore':'int64'})

In [64]:
final_df.dtypes

blue_baron_first           int64
blue_champion_first        int64
blue_dragon_first          int64
blue_inhibitor_first       int64
blue_riftherald_first      int64
blue_tower_first           int64
blue_win                   int64
red_baron_first            int64
red_champion_first         int64
red_dragon_first           int64
red_inhibitor_first        int64
red_riftherald_first       int64
red_tower_first            int64
red_win                    int64
blue_baron                 int64
blue_champion              int64
blue_dragon                int64
blue_inhibitor             int64
blue_riftherald            int64
blue_tower                 int64
red_baron                  int64
red_champion               int64
red_dragon                 int64
red_inhibitor              int64
red_riftherald             int64
red_tower                  int64
blue_lane_minion           int64
red_lane_minion            int64
blue_lane_gold             int64
red_lane_gold              int64
duration  

In [65]:
final_df.to_csv('data/Total_Combined_History.csv', index=False, encoding='utf-8')

In [66]:
df = pd.read_csv('data/Total_Combined_History.csv')

In [68]:
df.dtypes

blue_baron_first           int64
blue_champion_first        int64
blue_dragon_first          int64
blue_inhibitor_first       int64
blue_riftherald_first      int64
blue_tower_first           int64
blue_win                   int64
red_baron_first            int64
red_champion_first         int64
red_dragon_first           int64
red_inhibitor_first        int64
red_riftherald_first       int64
red_tower_first            int64
red_win                    int64
blue_baron                 int64
blue_champion              int64
blue_dragon                int64
blue_inhibitor             int64
blue_riftherald            int64
blue_tower                 int64
red_baron                  int64
red_champion               int64
red_dragon                 int64
red_inhibitor              int64
red_riftherald             int64
red_tower                  int64
blue_lane_minion           int64
red_lane_minion            int64
blue_lane_gold             int64
red_lane_gold              int64
duration  

In [12]:
pattern_gamemode = r"'gameMode':\s*'([^']+)'"
matches_gamemode = re.findall(pattern_gamemode, df_0_5000['info'][0])
matches_gamemode

['CLASSIC']

In [16]:
if matches_gamemode == ['CLASSIC']:
    print('YES')
else:
    print('NO')

YES


In [ ]:
def classic_game(df, i):
    pattern_gamemode = r"'gameMode':\s*'([^']+)'"
    matches_gamemode = re.findall(pattern_gamemode, df['info'][i])
    if matches_gamemode == ['CLASSIC']:
        


In [21]:
df_0_5000

,metadata,info,status
0,"{'dataVersion': '2', 'matchId': 'KR_6657198842...","{'gameCreation': 1692279451325, 'gameDuration'...",NaN
1,"{'dataVersion': '2', 'matchId': 'KR_6682314043...","{'gameCreation': 1693603792048, 'gameDuration'...",NaN
2,"{'dataVersion': '2', 'matchId': 'KR_6669781070...","{'gameCreation': 1692950197129, 'gameDuration'...",NaN
3,"{'dataVersion': '2', 'matchId': 'KR_6663488883...","{'gameCreation': 1692584355894, 'gameDuration'...",NaN
4,"{'dataVersion': '2', 'matchId': 'KR_6691019948...","{'gameCreation': 1694095693726, 'gameDuration'...",NaN
...,...,...,...
4995,"{'dataVersion': '2', 'matchId': 'KR_6681993445...","{'gameCreation': 1693582695269, 'gameDuration'...",NaN
4996,"{'dataVersion': '2', 'matchId': 'KR_6661082935...","{'gameCreation': 1692459997604, 'gameDuration'...",NaN
4997,"{'dataVersion': '2', 'matchId': 'KR_6659181463...","{'gameCreation': 1692376982267, 'gameDuration'...",NaN
4998,"{'dataVersion': '2', 'matchId': 'KR_6681436311...","{'gameCreation': 1693568349801, 'gameDuration'...",NaN


In [19]:
df_0_5000.drop(0)

,metadata,info,status
1,"{'dataVersion': '2', 'matchId': 'KR_6682314043...","{'gameCreation': 1693603792048, 'gameDuration'...",NaN
2,"{'dataVersion': '2', 'matchId': 'KR_6669781070...","{'gameCreation': 1692950197129, 'gameDuration'...",NaN
3,"{'dataVersion': '2', 'matchId': 'KR_6663488883...","{'gameCreation': 1692584355894, 'gameDuration'...",NaN
4,"{'dataVersion': '2', 'matchId': 'KR_6691019948...","{'gameCreation': 1694095693726, 'gameDuration'...",NaN
5,"{'dataVersion': '2', 'matchId': 'KR_6692874950...","{'gameCreation': 1694197485306, 'gameDuration'...",NaN
...,...,...,...
4995,"{'dataVersion': '2', 'matchId': 'KR_6681993445...","{'gameCreation': 1693582695269, 'gameDuration'...",NaN
4996,"{'dataVersion': '2', 'matchId': 'KR_6661082935...","{'gameCreation': 1692459997604, 'gameDuration'...",NaN
4997,"{'dataVersion': '2', 'matchId': 'KR_6659181463...","{'gameCreation': 1692376982267, 'gameDuration'...",NaN
4998,"{'dataVersion': '2', 'matchId': 'KR_6681436311...","{'gameCreation': 1693568349801, 'gameDuration'...",NaN


In [28]:
df = pd.concat([df_0_5000, df_5001_10000, df_10001_15000, df_15001_22000, df_22001_35000, df_35001_40000])

In [47]:
df.to_csv('total_match_result.csv', index=False, encoding='utf-8')

In [36]:
df

,metadata,info,status
0,"{'dataVersion': '2', 'matchId': 'KR_6657198842...","{'gameCreation': 1692279451325, 'gameDuration'...",NaN
1,"{'dataVersion': '2', 'matchId': 'KR_6682314043...","{'gameCreation': 1693603792048, 'gameDuration'...",NaN
2,"{'dataVersion': '2', 'matchId': 'KR_6669781070...","{'gameCreation': 1692950197129, 'gameDuration'...",NaN
3,"{'dataVersion': '2', 'matchId': 'KR_6663488883...","{'gameCreation': 1692584355894, 'gameDuration'...",NaN
4,"{'dataVersion': '2', 'matchId': 'KR_6691019948...","{'gameCreation': 1694095693726, 'gameDuration'...",NaN
...,...,...,...
4994,"{'dataVersion': '2', 'matchId': 'KR_6665972519...","{'gameCreation': 1692713860629, 'gameDuration'...",NaN
4995,"{'dataVersion': '2', 'matchId': 'KR_6659418905...","{'gameCreation': 1692399509718, 'gameDuration'...",NaN
4996,"{'dataVersion': '2', 'matchId': 'KR_6676313156...","{'gameCreation': 1693239847860, 'gameDuration'...",NaN
4997,"{'dataVersion': '2', 'matchId': 'KR_6672872228...","{'gameCreation': 1693069899124, 'gameDuration'...",NaN


In [66]:
game_modes = []
pattern = r"'gameMode':\s*'([^']+)'"
for i in range(len(df['info'])):
    info_value = df['info'][i]
    if pd.isna(info_value):
        game_modes.append('Unknown')  # NaN 값 처리
        continue
    if isinstance(info_value, str):
        match = re.search(pattern, info_value)
        if match:
            game_mode = match.group(1)
        else:
            game_mode = 'Unknown'
        game_modes.append(game_mode)

df['game_mode'] = game_modes

In [67]:
classic_count = df['game_mode'].value_counts().get('CLASSIC', 0)
print(f"클래식 게임 수: {classic_count}")

클래식 게임 수: 33199


In [60]:
df = pd.read_csv('data/total_match_result.csv')

In [70]:
classic_df = df[df['game_mode'] == 'CLASSIC']

In [71]:
classic_df.to_csv('data/total_classic_match_result.csv', index=False, encoding='utf-8')

In [75]:
classic_df.reset_index(drop=True, inplace=True)

In [76]:
classic_df

,metadata,info,status,game_mode
0,"{'dataVersion': '2', 'matchId': 'KR_6657198842...","{'gameCreation': 1692279451325, 'gameDuration'...",NaN,CLASSIC
1,"{'dataVersion': '2', 'matchId': 'KR_6682314043...","{'gameCreation': 1693603792048, 'gameDuration'...",NaN,CLASSIC
2,"{'dataVersion': '2', 'matchId': 'KR_6669781070...","{'gameCreation': 1692950197129, 'gameDuration'...",NaN,CLASSIC
3,"{'dataVersion': '2', 'matchId': 'KR_6663488883...","{'gameCreation': 1692584355894, 'gameDuration'...",NaN,CLASSIC
4,"{'dataVersion': '2', 'matchId': 'KR_6692874950...","{'gameCreation': 1694197485306, 'gameDuration'...",NaN,CLASSIC
...,...,...,...,...
33194,"{'dataVersion': '2', 'matchId': 'KR_6656071489...","{'gameCreation': 1692206082449, 'gameDuration'...",NaN,CLASSIC
33195,"{'dataVersion': '2', 'matchId': 'KR_6659418905...","{'gameCreation': 1692399509718, 'gameDuration'...",NaN,CLASSIC
33196,"{'dataVersion': '2', 'matchId': 'KR_6676313156...","{'gameCreation': 1693239847860, 'gameDuration'...",NaN,CLASSIC
33197,"{'dataVersion': '2', 'matchId': 'KR_6672872228...","{'gameCreation': 1693069899124, 'gameDuration'...",NaN,CLASSIC


In [77]:
def teams_preprocessing(df, i):
    
    # first kill 목록
    tf = ['blue_baron_first', 'blue_champion_first', 'blue_dragon_first', 'blue_inhibitor_first', 'blue_riftherald_first', 'blue_tower_first', 'blue_win',
              'red_baron_first', 'red_champion_first', 'red_dragon_first', 'red_inhibitor_first', 'red_riftherald_first', 'red_tower_first', 'red_win']

    # total kill 목록
    kill = ['blue_baron', 'blue_champion', 'blue_dragon', 'blue_inhibitor', 'blue_riftherald', 'blue_tower',
            'red_baron', 'red_champion', 'red_dragon', 'red_inhibitor', 'red_riftherald', 'red_tower']

    # 기타 컬럼 목록
    others = ['blue_lane_minion', 'red_lane_minion', 'blue_lane_gold', 'red_lane_gold', 'duration', 'blue_gold_perM', 'red_gold_perM', 'blue_gold_earned', 
             'red_gold_earned', 'blue_kda', 'red_kda', 'blue_solo_kill', 'red_solo_kill', 'blue_visionscore_perM', 'red_visionscore_perM', 'blue_level', 
             'red_level', 'blue_death', 'red_death', 'blue_damage', 'red_damage', 'blue_jungle_minion', 'red_jungle_minion', 'blue_total_minion',
             'red_total_minion', 'blue_death_time', 'red_death_time', 'blue_visionscore', 'red_visionscore']

    # 추가: df['info'][i] 타입 확인
    if pd.isna(df['info'][i]):
        print(f"Skipping index {i}")
        return None  # 혹은 빈 DataFrame 반환
    

    # teams 데이터 불러오기
    teams = df['info'][i][df['info'][i].find('teams'):]

    # first kill
    pattern_tf = r'(True|False)'
    matches_tf = re.findall(pattern_tf, teams)

    # total kill
    pattern_kills = r"'kills': ([0-9]*)"
    matches_kill = re.findall(pattern_kills, teams)

    # laneMinionsFirst10Minutes
    pattern_laneminion = r"'laneMinionsFirst10Minutes': ([0-9]*)"
    matches_laneminion = re.findall(pattern_laneminion, df['info'][i])
    matches_laneminion = [sum(map(int, matches_laneminion[:5])), sum(map(int, matches_laneminion[5:]))]

    # laningPhaseGoldExpAdvantage
    pattern_lanegold = r"'laningPhaseGoldExpAdvantage': ([0-9]*)"
    matches_lanegold = re.findall(pattern_lanegold, df['info'][i])
    matches_lanegold = [sum(map(int, matches_lanegold[:5])), sum(map(int, matches_lanegold[5:]))]

    # gameDuration
    pattern_gameduration = r"'gameDuration': ([0-9]*)"
    matches_gameduration = re.findall(pattern_gameduration, df['info'][i])
    matches_gameduration = list(map(int, matches_gameduration))

    # golPerMinute
    pattern_gold_perM = r"'goldPerMinute': ([0-9]*\.?[0-9]*)"
    matches_gold_perM = re.findall(pattern_gold_perM, df['info'][i])
    matches_gold_perM = [round(sum(map(float, matches_gold_perM[:5]))/5, 2), round(sum(map(float, matches_gold_perM[5:]))/5, 2)]

    # goldEarned
    pattern_goldearned = r"'goldEarned': ([0-9]*)"
    matches_goldearned = re.findall(pattern_goldearned, df['info'][i])
    matches_goldearned = [sum(map(int, matches_goldearned[:5])), sum(map(int, matches_goldearned[5:]))]

    # kda
    pattern_kda = r"'kda': ([0-9]*\.?[0-9]*)"
    matches_kda = re.findall(pattern_kda, df['info'][i])
    matches_kda = [round(sum(map(float, matches_kda[:5]))/5, 2), round(sum(map(float, matches_kda[5:]))/5, 2)]

    # soloKills
    pattern_solokill = r"'soloKills': ([0-9]*)"
    matches_solokill = re.findall(pattern_solokill, df['info'][i])
    matches_solokill = [sum(map(int, matches_solokill[:5])), sum(map(int, matches_solokill[5:]))]

    # visionScorePerMinute
    pattern_visionScore_perM = r"'visionScorePerMinute': ([0-9]*\.?[0-9]*)"
    matches_visionScore_perM = re.findall(pattern_visionScore_perM, df['info'][i])
    matches_visionScore_perM = [round(sum(map(float, matches_visionScore_perM[:5]))/5, 2), round(sum(map(float, matches_visionScore_perM[5:]))/5, 2)]

    # champLevel
    pattern_champLevel = r"'champLevel': ([0-9]*)"
    matches_champLevel = re.findall(pattern_champLevel, df['info'][i])
    matches_champLevel = [sum(map(int, matches_champLevel[:5]))/5, sum(map(int, matches_champLevel[5:]))/5]

    # deaths
    pattern_deaths = r"'deaths': ([0-9]*)"
    matches_deaths = re.findall(pattern_deaths, df['info'][i])
    matches_deaths = [sum(map(int, matches_deaths[:5])), sum(map(int, matches_deaths[5:]))]

    # totalDamageDealtToChampions
    pattern_damage = r"'totalDamageDealtToChampions': ([0-9]*)"
    matches_damage = re.findall(pattern_damage, df['info'][i])
    matches_damage = [sum(map(int, matches_damage[:5]))/5, sum(map(int, matches_damage[5:]))/5]

    # totalEnemyJungleMinionsKilled
    pattern_jungle_minion = r"'totalEnemyJungleMinionsKilled': ([0-9]*)"
    matches_jungle_minion = re.findall(pattern_jungle_minion, df['info'][i])
    matches_jungle_minion = [sum(map(int, matches_jungle_minion[:5])), sum(map(int, matches_jungle_minion[5:]))]

    # totalMinionsKilled
    pattern_total_minion = r"'totalMinionsKilled': ([0-9]*)"
    matches_total_minion = re.findall(pattern_total_minion, df['info'][i])
    matches_total_minion = [sum(map(int, matches_total_minion[:5])), sum(map(int, matches_total_minion[5:]))]

    # totalTimeSpentDead
    pattern_death_time = r"'totalTimeSpentDead': ([0-9]*)"
    matches_death_time = re.findall(pattern_death_time, df['info'][i])
    matches_death_time = [sum(map(int, matches_death_time[:5])), sum(map(int, matches_death_time[5:]))]

    # visionScore
    pattern_visionscore = r"'visionScore': ([0-9]*)"
    matches_visionscore = re.findall(pattern_visionscore, df['info'][i])
    matches_visionscore = [sum(map(int, matches_visionscore[:5])), sum(map(int, matches_visionscore[5:]))]

    # 데이터 합치기
    combined_data = dict(zip(tf + kill + others, matches_tf + matches_kill + matches_laneminion + matches_lanegold + 
                             matches_gameduration + matches_gold_perM + matches_goldearned + matches_kda + matches_solokill + matches_visionScore_perM +
                             matches_champLevel + matches_deaths + matches_damage + matches_jungle_minion + matches_total_minion + matches_death_time +
                             matches_visionscore))

    # 데이터 프레임
    match_history = pd.DataFrame([combined_data])
    return match_history

all_match_history = pd.DataFrame()

for i in range(len(classic_df)):
    result = teams_preprocessing(classic_df, i)
    if result is not None:  # 추가: 반환된 값이 None이 아닐 경우만 concat
        all_match_history = pd.concat([all_match_history, result])

In [79]:
all_match_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33199 entries, 0 to 0
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   blue_baron_first       33199 non-null  object 
 1   blue_champion_first    33199 non-null  object 
 2   blue_dragon_first      33199 non-null  object 
 3   blue_inhibitor_first   33199 non-null  object 
 4   blue_riftherald_first  33199 non-null  object 
 5   blue_tower_first       33199 non-null  object 
 6   blue_win               33199 non-null  object 
 7   red_baron_first        33199 non-null  object 
 8   red_champion_first     33199 non-null  object 
 9   red_dragon_first       33199 non-null  object 
 10  red_inhibitor_first    33199 non-null  object 
 11  red_riftherald_first   33199 non-null  object 
 12  red_tower_first        33199 non-null  object 
 13  red_win                33199 non-null  object 
 14  blue_baron             33199 non-null  object 
 15  blue_c

In [80]:
all_match_history

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,False,False,False,False,False,False,False,False,True,True,...,3701.2,8244.6,4,27,333,452,358,66,54,78
0,False,False,False,False,True,False,False,True,True,True,...,12938.8,15635.2,12,45,552,581,833,318,127,153
0,True,False,True,True,True,True,True,False,True,False,...,11686.4,10214.8,22,6,548,580,326,652,138,93
0,False,True,True,True,True,False,False,True,False,False,...,38695.6,30668.8,34,19,865,911,1344,1575,288,259
0,True,False,False,True,True,True,True,False,True,True,...,27495.4,22866.0,69,0,653,576,998,992,196,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,True,False,False,False,True,False,False,False,True,True,...,13914.0,18558.2,12,23,693,679,703,591,191,201
0,False,True,True,False,True,False,True,False,False,False,...,7996.0,7145.8,9,0,402,379,104,329,70,50
0,False,False,False,False,False,False,False,False,True,False,...,2365.4,4105.6,4,6,113,284,140,0,23,30
0,False,False,False,False,True,False,False,False,True,True,...,8103.0,11631.8,12,32,360,497,564,328,83,91


In [81]:
all_match_history.isnull().sum()

blue_baron_first         0
blue_champion_first      0
blue_dragon_first        0
blue_inhibitor_first     0
blue_riftherald_first    0
blue_tower_first         0
blue_win                 0
red_baron_first          0
red_champion_first       0
red_dragon_first         0
red_inhibitor_first      0
red_riftherald_first     0
red_tower_first          0
red_win                  0
blue_baron               0
blue_champion            0
blue_dragon              0
blue_inhibitor           0
blue_riftherald          0
blue_tower               0
red_baron                0
red_champion             0
red_dragon               0
red_inhibitor            0
red_riftherald           0
red_tower                0
blue_lane_minion         0
red_lane_minion          0
blue_lane_gold           0
red_lane_gold            0
duration                 0
blue_gold_perM           0
red_gold_perM            0
blue_gold_earned         0
red_gold_earned          0
blue_kda                 0
red_kda                  0
b

In [ ]:
all_match_history

In [84]:
all_match_history = all_match_history.replace({'True': 1, 'False': 0})

In [85]:
all_match_history

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,0,0,0,0,0,0,0,0,1,1,...,3701.2,8244.6,4,27,333,452,358,66,54,78
0,0,0,0,0,1,0,0,1,1,1,...,12938.8,15635.2,12,45,552,581,833,318,127,153
0,1,0,1,1,1,1,1,0,1,0,...,11686.4,10214.8,22,6,548,580,326,652,138,93
0,0,1,1,1,1,0,0,1,0,0,...,38695.6,30668.8,34,19,865,911,1344,1575,288,259
0,1,0,0,1,1,1,1,0,1,1,...,27495.4,22866.0,69,0,653,576,998,992,196,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,0,0,0,1,0,0,0,1,1,...,13914.0,18558.2,12,23,693,679,703,591,191,201
0,0,1,1,0,1,0,1,0,0,0,...,7996.0,7145.8,9,0,402,379,104,329,70,50
0,0,0,0,0,0,0,0,0,1,0,...,2365.4,4105.6,4,6,113,284,140,0,23,30
0,0,0,0,0,1,0,0,0,1,1,...,8103.0,11631.8,12,32,360,497,564,328,83,91


In [86]:
all_match_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33199 entries, 0 to 0
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   blue_baron_first       33199 non-null  int64  
 1   blue_champion_first    33199 non-null  int64  
 2   blue_dragon_first      33199 non-null  int64  
 3   blue_inhibitor_first   33199 non-null  int64  
 4   blue_riftherald_first  33199 non-null  int64  
 5   blue_tower_first       33199 non-null  int64  
 6   blue_win               33199 non-null  int64  
 7   red_baron_first        33199 non-null  int64  
 8   red_champion_first     33199 non-null  int64  
 9   red_dragon_first       33199 non-null  int64  
 10  red_inhibitor_first    33199 non-null  int64  
 11  red_riftherald_first   33199 non-null  int64  
 12  red_tower_first        33199 non-null  int64  
 13  red_win                33199 non-null  int64  
 14  blue_baron             33199 non-null  object 
 15  blue_c

In [87]:
all_match_history['blue_baron']             

0    0
0    0
0    1
0    0
0    2
    ..
0    1
0    0
0    0
0    0
0    3
Name: blue_baron, Length: 33199, dtype: object

In [88]:
all_match_history = all_match_history.astype({'blue_baron':'int64', 'blue_champion':'int64', 'blue_dragon':'int64', 'blue_inhibitor':'int64',
                            'blue_riftherald':'int64', 'blue_tower':'int64', 'red_baron':'int64', 'red_champion':'int64',
                           'red_dragon':'int64', 'red_inhibitor':'int64', 'red_riftherald':'int64', 'red_tower':'int64'})

In [89]:
all_match_history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33199 entries, 0 to 0
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   blue_baron_first       33199 non-null  int64  
 1   blue_champion_first    33199 non-null  int64  
 2   blue_dragon_first      33199 non-null  int64  
 3   blue_inhibitor_first   33199 non-null  int64  
 4   blue_riftherald_first  33199 non-null  int64  
 5   blue_tower_first       33199 non-null  int64  
 6   blue_win               33199 non-null  int64  
 7   red_baron_first        33199 non-null  int64  
 8   red_champion_first     33199 non-null  int64  
 9   red_dragon_first       33199 non-null  int64  
 10  red_inhibitor_first    33199 non-null  int64  
 11  red_riftherald_first   33199 non-null  int64  
 12  red_tower_first        33199 non-null  int64  
 13  red_win                33199 non-null  int64  
 14  blue_baron             33199 non-null  int64  
 15  blue_c

In [91]:
all_match_history.to_csv('Final_all_match_hisotry.csv', index=False, encoding='utf-8')

In [93]:
df = pd.read_csv('Final_all_match_hisotry.csv')

In [94]:
df

,blue_baron_first,blue_champion_first,blue_dragon_first,blue_inhibitor_first,blue_riftherald_first,blue_tower_first,blue_win,red_baron_first,red_champion_first,red_dragon_first,...,blue_damage,red_damage,blue_jungle_minion,red_jungle_minion,blue_total_minion,red_total_minion,blue_death_time,red_death_time,blue_visionscore,red_visionscore
0,0,0,0,0,0,0,0,0,1,1,...,3701.2,8244.6,4,27,333,452,358,66,54,78
1,0,0,0,0,1,0,0,1,1,1,...,12938.8,15635.2,12,45,552,581,833,318,127,153
2,1,0,1,1,1,1,1,0,1,0,...,11686.4,10214.8,22,6,548,580,326,652,138,93
3,0,1,1,1,1,0,0,1,0,0,...,38695.6,30668.8,34,19,865,911,1344,1575,288,259
4,1,0,0,1,1,1,1,0,1,1,...,27495.4,22866.0,69,0,653,576,998,992,196,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33194,1,0,0,0,1,0,0,0,1,1,...,13914.0,18558.2,12,23,693,679,703,591,191,201
33195,0,1,1,0,1,0,1,0,0,0,...,7996.0,7145.8,9,0,402,379,104,329,70,50
33196,0,0,0,0,0,0,0,0,1,0,...,2365.4,4105.6,4,6,113,284,140,0,23,30
33197,0,0,0,0,1,0,0,0,1,1,...,8103.0,11631.8,12,32,360,497,564,328,83,91


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33199 entries, 0 to 33198
Data columns (total 55 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   blue_baron_first       33199 non-null  int64  
 1   blue_champion_first    33199 non-null  int64  
 2   blue_dragon_first      33199 non-null  int64  
 3   blue_inhibitor_first   33199 non-null  int64  
 4   blue_riftherald_first  33199 non-null  int64  
 5   blue_tower_first       33199 non-null  int64  
 6   blue_win               33199 non-null  int64  
 7   red_baron_first        33199 non-null  int64  
 8   red_champion_first     33199 non-null  int64  
 9   red_dragon_first       33199 non-null  int64  
 10  red_inhibitor_first    33199 non-null  int64  
 11  red_riftherald_first   33199 non-null  int64  
 12  red_tower_first        33199 non-null  int64  
 13  red_win                33199 non-null  int64  
 14  blue_baron             33199 non-null  int64  
 15  bl